In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import multiprocessing.pool

# Basic SAW generator
Does not generate SAWs uniformly

In [2]:
DIRECTIONS = np.array([[1, 0], [0, 1], [-1, 0], [0, -1]])

In [3]:
def generate_valid_saw(L):
    while True:
        coords = [np.array([0, 0])]
        visited = set((0, 0))

        while len(coords) <= L:
            neighbours = [move + coords[-1] for move in DIRECTIONS if tuple(move + coords[-1]) not in visited]
            if len(neighbours) == 0: # no possible moves, keep still
                break
            coords.append(random.choice(neighbours))
            visited.add(tuple(coords[-1]))
        if len(coords) == L + 1:
            return np.array(coords)


In [4]:
generate_valid_saw(10)

array([[0, 0],
       [1, 0],
       [1, 1],
       [1, 2],
       [0, 2],
       [0, 3],
       [1, 3],
       [2, 3],
       [2, 4],
       [2, 5],
       [1, 5]])

# Recursive SAW formulation

Let $B(L_A, L_B)$ be the probability that when sampling SAWs of length $L_A$ and $L_B$ independently and uniformly at random, their concatenation is still a SAW.

$B(L_A, L_B) = {c_{L_A + L_B} \over c_{L_A} c_{L_B}}$

$\mu^{L_A} \approx {c_{L_A + L_B} \over c_{L_B}} = B(L_A, L_B) c_{L_A}$

We can generate samples of SAWs of length $L_A$ and $L_B$ to estimate $B(L_A, L_B)$

In [5]:
# Returns True if z_A can be concatenated with z_B
# such that the resulting SAW is valid
def can_concatenate(z_A, z_B):
    z = np.concat((z_A, z_B[1:]+z_A[-1])).tolist()
    return len(z) == len(set(map(tuple, z)))


def estimate_B(Z_A, Z_B):
    if len(Z_A) != len(Z_B):
        raise ValueError("Z_A and Z_B must have the same length")

    Z = zip(Z_A, Z_B)

    successes = 0
    for z_A, z_B in Z:
        if can_concatenate(z_A, z_B):
            successes += 1

    return successes / len(Z_A)


# estimates c_{L_A + L_B}
def estimate_c(Z_A, Z_B, c_L_A, c_L_B):
    return estimate_B(Z_A, Z_B) * c_L_A * c_L_B


def estimate_mu(C):
    mus = []
    for c1, c2 in zip(C[:-1], C[1:]):
        mus.append((c2[1] / c1[1]) ** (1 / c1[0]))
    return mus


# Generating SAWs with Pivot Algorithm

In [6]:
def pivot(z, pivot_generator, transform_generator):
    pivot_idx = pivot_generator(len(z)-1)
    transform = transform_generator(z)
    
    before = z[:pivot_idx + 1]
    pivot = z[pivot_idx]
    after = z[pivot_idx + 1:] - pivot
    z_new = np.vstack((before, (after @ transform.T) + pivot))

    if len(set(map(tuple, z_new))) != len(z_new):
        return z, False
    return z_new, True


def generate_saw_pivot(z, pivot_generator, transform_generator):
    successes = 0
    while successes < len(z):
        z, success = pivot(z, pivot_generator, transform_generator)
        if success:
            successes += 1
    return z


def saw_pivot_batch_generator(L, n, walk_generator, pivot_generator, transform_generator):
    z = walk_generator(L)
    Z = []
    for _ in range(L):
        z = generate_saw_pivot(z, pivot_generator, transform_generator)
    for _ in range(n):
        z = generate_saw_pivot(z, pivot_generator, transform_generator)
        Z.append(z)
    return Z


# Transformation generator

In [7]:
TRANSFORMS = [
    np.array([[1, 0], [0, 1]]), # identity
    np.array([[-1, 0], [0, 1]]), # reflect over y-axis
    np.array([[1, 0], [0, -1]]), # reflect over x-axis
    np.array([[-1, 0], [0, -1]]), # rotate 180 degrees
    np.array([[0, 1], [1, 0]]), # reflect over y=x
    np.array([[0, -1], [1, 0]]), # rotate 90 degrees counterclockwise
    np.array([[0, 1], [-1, 0]]), # rotate 90 degrees clockwise
    np.array([[0, -1], [-1, 0]]) # reflect over y=-x
]

In [8]:
# z is a param for more complex transform generators
# that take the current state of the walk into account
def naive_transform_generator(z):
    return random.choice(TRANSFORMS)


# Pivot generator

In [9]:
def uniform_pivot_generator(L):
    return random.randint(1, L)


def gaussian_pivot_generator(L):
    mean = (L + 1) / 2
    stddev = L / 4
    pivot = round(np.random.normal(mean, stddev))
    if pivot < 1:
        return 1
    elif pivot > L:
        return L
    return pivot


# Could maybe try adaptive methods, but need some sort of acceptance method to ensure uniform sampling

# Speed comparison

In [10]:
estimate_c(saw_pivot_batch_generator(10, 10_000, generate_valid_saw, uniform_pivot_generator, naive_transform_generator),\
           saw_pivot_batch_generator(10, 10_000, generate_valid_saw, uniform_pivot_generator, naive_transform_generator),\
           44100, 44100)

965209203.0000001

In [11]:
estimate_c(saw_pivot_batch_generator(10, 10_000, generate_valid_saw, gaussian_pivot_generator, naive_transform_generator),\
           saw_pivot_batch_generator(10, 10_000, generate_valid_saw, gaussian_pivot_generator, naive_transform_generator),\
           44100, 44100)

901419435.0000001

# Multi-threading

In [12]:
def generate_saw_pivot_batch(L, n, walk_generator, pivot_generator, transform_generator, batch_size=1000):
    def generator(no_samples):
        return saw_pivot_batch_generator(L, no_samples, walk_generator, pivot_generator, transform_generator)
    
    thread_pool = multiprocessing.pool.ThreadPool(processes=10)
    batches = [batch_size] * (n // batch_size) + ([n % batch_size] if n % batch_size != 0 else [])
    return np.vstack(thread_pool.map(generator, batches))
    

In [13]:
estimate_c(generate_saw_pivot_batch(10, 10_000, generate_valid_saw, uniform_pivot_generator, naive_transform_generator),
           generate_saw_pivot_batch(10, 10_000, generate_valid_saw, uniform_pivot_generator, naive_transform_generator),
           44100, 44100)

910754523.0

# Estimation

In [ ]:
C_pivot = [(10, 44100)]
n = 10_000
stopping_condition = 640 # stops when the last L is less than this value

while C_pivot[-1][0] < stopping_condition:
    L, c_L = C_pivot[-1]
    print(f'Working on L = {2*L}...')
    Z_A = generate_saw_pivot_batch(L, n, generate_valid_saw, uniform_pivot_generator, naive_transform_generator)
    Z_B = generate_saw_pivot_batch(L, n, generate_valid_saw, uniform_pivot_generator, naive_transform_generator)
    C_pivot.append((L*2, estimate_c(Z_A, Z_B, c_L, c_L)))

Working on L = 20...
Working on L = 40...
Working on L = 80...
Working on L = 160...
Working on L = 320...


In [ ]:
C_pivot

[(10, 44100),
 (20, 901030473.0),
 (40, 3.049330810259412e+17),
 (80, 2.7755778895336004e+34),
 (160, 1.8635571109879196e+68)]

In [ ]:
estimate_mu(C_pivot)

[2.6979263448013406, 2.669766339356766, 2.6544755039330816, 2.647508518178019]

In [ ]:
X = np.array([L for L, _ in C_pivot])
Y1 = np.array([c_L for _, c_L in C_pivot])
Y2 = np.array([mu for mu in estimate_mu(C_pivot)])

# Visualisation

In [ ]:
plt.plot(X, Y1)
plt.title("Pivot Algorithm c_L (log scale)")
plt.xlabel("L")
plt.yscale("log")
plt.ylabel("c_L")
plt.savefig("diagrams/pivot1.png")

In [ ]:
plt.plot(X, Y2)
plt.title("Pivot Algorithm mu")
plt.xlabel("L")
plt.ylabel("mu")
plt.savefig("diagrams/pivot2.png")